# Imports

## Equality

````
module code.eq where

infix 4 _≡_
data _≡_ {A : Set} (x : A) : A → Set where
  refl : x ≡ x
  
sym : {A : Set} {x y : A} → x ≡ y → y ≡ x
sym refl = refl

trans : {A : Set} {x y z : A} → x ≡ y → y ≡ z → x ≡ z
trans refl refl = refl

cong : {A B : Set} (f : A → B) {x y : A} → x ≡ y → f x ≡ f y
cong f refl = refl
````

## Natural numbers

````
module code.nat where

data ℕ : Set where 
  zero : ℕ
  suc : ℕ → ℕ
  
{-# BUILTIN NATURAL ℕ #-}

infixl 5 _+_

_+_ : ℕ → ℕ → ℕ
zero + m = m
(suc n) + m = suc (n + m)
  
data _≤_ : ℕ → ℕ → Set where
  0≤n : {n : ℕ} → 0 ≤ n
  s≤s : {m n : ℕ} → m ≤ n → suc m ≤ suc n
````

## Dependent product

````
module code.product where

infixr 4 _,_
record Σ {l} (A : Set l) (B : A → Set) : Set l where
    constructor _,_
    field
      fst : A
      snd : B fst

-- non-dependent product
infixr 2 _∧_
_∧_ : Set → Set → Set
A ∧ B = Σ A (λ _ → B)

infix 0 thereExists
thereExists : ∀ {l} {A : Set l} (B : A → Set) → Set l
thereExists {_} {A} B = Σ A B

syntax thereExists (λ x → B) = ∃[ x ] B
````

## Negation

````
module code.neg where

data ⊥ : Set where

⊥-elim : {A : Set} → ⊥ → A
⊥-elim ()

infix 3 ¬_ -- higher priority than ∨ and ∧

¬_ : Set → Set
¬ A = A → ⊥
````

## Boolean values

````
module code.bool where
````

We define the domain of boolean values,
which will be useful later in order to give semantics to boolean expressions.

````
data 𝔹 : Set where
  true : 𝔹
  false : 𝔹
````

### **Exercise**: Computation on `𝔹`

Define the semantic operations of conjunction, disjunction, and negation on boolean values.

In [162]:
{-# OPTIONS --allow-unsolved-metas #-}
module code.bool.op where
open import code.bool public

infix 5 _&&_
_&&_ : 𝔹 → 𝔹 → 𝔹
b && c = ?

infix 6 _||_
_||_ : 𝔹 → 𝔹 → 𝔹
b || c = ?

infix 4 ~_
~_ : 𝔹 → 𝔹
~ b = ?

# Decidables

## Order on `ℕ`

Consider again the inductive definition of `≤` for natural numbers:

```agda
--data _≤_ : ℕ → ℕ → Set where
--0≤n : {n : ℕ} → 0 ≤ n
--s≤s : {m n : ℕ} → m ≤ n → suc m ≤ suc n
```

<!--The essence of this definition is that evidence of `m ≤ n`
can be used to produce evidence of `suc m ≤ suc n`. -->

While the definition of `≤` can be used to establish `m ≤ n`
for concrete numbers `m` and `n`,
it does not directly yield an algorithm for checking whether `m ≤ n` holds.

(And for a good reason. As we will see below, inductive definitions such as `≤` can encode two-counter Minsky machines, and therefore such relations are undecidable in general.)

### **Exercise**: `≤ᵇ`

Write a program that returns `true` or `false` depending on whether `m ≤ n` holds or not.

*Hint:* Proceed by recursion on the first argument.

In [163]:
{-# OPTIONS --allow-unsolved-metas #-}
module code.bool.leq where
open import code.nat public
open import code.bool public

_≤ᵇ_ : ℕ → ℕ → 𝔹
m ≤ᵇ n = ?

?0 : 𝔹


The function `_≤ᵇ_` *computes* the answer to whether `m ≤ n` holds.
This means that `m ≤ n` is a *decidable* property of the natural numbers.

However, `_≤ᵇ_` does not itself compute a *proof* that `m ≤ n` holds,
and in fact there may well be bugs in the definition of `_≤ᵇ_`.

## Decidable properties

We show that we can develop `_≤ᵇ_` and the proof of its correctness in a single definition.

````
module code.decidable where
open import code.neg public
````

We can combine a decision procedure for a property
together with the construction of the evidence that the property holds
with the introduction of the class of *decidable properties*.

````
data Dec (A : Set) : Set where
  yes : A → Dec A
  no : ¬ A → Dec A
````

The intuition is that evidence for `Dec A` has one of two possible forms:
Either it is of the form `yes p` where `p` is a proof that `A` holds,
or it is of the form `no ¬p` where `¬p` is a proof that `¬ A` holds.

The crucial improvement is the introduction of evidence also when the property does not hold.

### **Exercise:** `_≤_` is decidable

Show that the order `_≤_` on natural number is decidable.
This is an instance of *internal verification*,
whereby an algorithm and its proof of correctness are developed together.

In [ ]:
module code.nat.leq.decidable where
open import code.decidable public
open import code.nat public

≤-suc : {m n : ℕ} → suc m ≤ suc n → m ≤ n
≤-suc = ?

_≤?_ : (m n : ℕ) → Dec (m ≤ n)
0 ≤? _ = ?
(suc _) ≤? 0 = ?
(suc m) ≤? (suc n) with m ≤? n
... | yes m≤n = ?
... | no ¬m≤n = ?

?0 : suc m ≤ suc n → m ≤ n
?1 : Dec (zero ≤ n)
?2 : Dec (suc x ≤ zero)
?3 : Dec (suc m ≤ suc n)
?4 : Dec (suc m ≤ suc n)


# Arithmetic expressions

````
{-# OPTIONS --allow-unsolved-metas #-}

module code.aexp where
open import code.eq public
open import code.nat public
open import code.decidable public
open import code.product public
````

We develop an eager denotational and operational semantics
for a simple language of arithmetic expressions,
and we prove that they agree.

## Variables

We represent *variable names* as natural number.
Any countable domain with decidable equality (such as strings) would work here

````
Var = ℕ
````

### Decidable equality

We  show that variables have a decidable equality,
which boils down to show that the same property holds for natural numbers.

````
var-inv : ∀ {x y} → suc x ≡ suc y → x ≡ y
var-inv refl = refl

infix 5 _≡Var?_
_≡Var?_ : (x y : Var) → Dec (x ≡ y)
0 ≡Var? 0 = yes refl
0 ≡Var? suc _ = no λ ()
suc _ ≡Var? 0 = no λ ()
suc x ≡Var? suc y with x ≡Var? y
... | yes refl = yes refl
... | no neq = no λ eq → neq (var-inv eq)
````

## Syntax of expressions

We define a minimalistic language of arithmetic expressions
comprising variables and a let assignment construct.

````
data AExp : Set where
  num : ℕ → AExp
  var : Var → AExp
  plus : AExp → AExp → AExp
  let-exp : Var → AExp → AExp → AExp
````

For example,
the following expression adds one to a variable (called 10).
This is pure syntax so far, no calculation is being performed.

````
add-one : AExp
add-one = plus (var 10) (num 1)
````

## Environments

In order to represent the value of free variables,
we use environments.

````
Env = Var → ℕ
````

The following environment assigns value `200` to the variable named `10`,
and value `40` to every other variable.

````
ρ0 : Env
ρ0 10 = 200
ρ0 _ = 40
````

### **Exercise**: `Env` updates

Given an environment `ρ`, a variable `x` and a new value  `m` for this variable,
let `ρ [ x ↦ m ]` be a new environment which assigns `m` to `x`,
and agrees with `ρ` elsewhere.

Complete the definition for the environment update function `_[_↦_]`.

In [ ]:
{-# OPTIONS --allow-unsolved-metas #-}
module code.aexp.env where
open import code.aexp public

infixr 30 _[_↦_]
_[_↦_] : Env → Var → ℕ → Env
(ρ [ x ↦ m ]) y with x ≡Var? y
... | yes _ = ?
... | no _ = ?

?0 : ℕ
?1 : ℕ


## **Exercise**: `⟦_⟧A_`

We are now ready to give denotational semantics to arithmetic expressions.
Complete the definition of `⟦_⟧A_` below.

In [ ]:
{-# OPTIONS --allow-unsolved-metas #-}
module code.aexp.sem where
open import code.aexp.env public

infix 10 ⟦_⟧A_
⟦_⟧A_ : AExp → Env → ℕ
⟦ e ⟧A ρ = ?

?0 : ℕ


# Boolean expressions

````
module code.bexp where
open import code.aexp.sem public
open import code.nat.leq.decidable public
open import code.bool.op public
````

We define a simple language of boolan expressions.

## Syntax of expressions

An element in `BExp` is a boolean combination
of atomic expressions of the form `leq e f`,
where `e` and `f` are arithmetic expressions.

````
data BExp : Set where
  tt : BExp
  ff : BExp
  and : BExp → BExp → BExp
  or : BExp → BExp → BExp
  not : BExp → BExp
  leq : AExp → AExp → BExp
````

## **Exercise**: `⟦_⟧B_`

 Define the denotational semantics of boolean expressions.
 
 *Hint:* In the `leq` case you will need `_≤?_`.

In [ ]:
{-# OPTIONS --allow-unsolved-metas #-}
module code.bexp.sem where
open import code.bexp public

infix 10 ⟦_⟧B_
⟦_⟧B_ : BExp → Env → 𝔹
⟦ b ⟧B ρ = ?

?0 : 𝔹


# Imperative programs

````
module code.imp where
open import code.bexp.sem public

infixr 20 _⨟_
infix 25 _≔_
````

## Syntax of programs 

We define a simple imperative language,
leveraging on arithmetic and boolean expressions defined above.

````
data Cmd : Set where
  skip : Cmd
  _≔_ : Var → AExp → Cmd
  _⨟_ : Cmd → Cmd → Cmd
  if_then_else_ : BExp → Cmd → Cmd → Cmd
  while_do:_ : BExp → Cmd → Cmd
````

For example, the following is a valid program (piece of syntax).

````
loop : Cmd
loop = while tt do: skip
````

(We added a colon in the syntax of `while_do:_`
because `do` is a reserved keyword.)

## Denotational semantics

It may come as a surprise,
but it is not possible to define the semantics of imperative programs as an Agda function,
because Agda allows only terminating (i.e., total) functions.
On the other hand, imperative programs, such as `loop` above, may not terminate.

Therefore, we resort to an operational semantics.

## Big-steps operational semantics

We begin with the notion of state,
which is the same as environments as of now,
but needs not be.

````
State = Var → ℕ
````

The definition of the operational semantics of imperative programs
follows a case analysis.

````
infix 4 _,_⇒_
data _,_⇒_ : Cmd → State → State → Set where
````

### `skip`

The `skip` command terminates immediately without changing the state.

````
  ⇒-skip : ∀ {s} →
    skip , s ⇒ s
````

### `x ≔ e`

The assignment command modifies the state
by updating the value of `x` to the value of `e` in the current state.

````
  ⇒-assign : ∀ {s x e} →
    x ≔ e , s ⇒ s [ x ↦ ⟦ e ⟧A s ]
````

### `c ⨟ d`

Sequencing two commands amounts to thread the state change.

````
  ⇒-seq : ∀ {c d s s' s''} →
    c , s ⇒ s' →
    d , s' ⇒ s'' →
    c ⨟ d , s ⇒ s''
````

### `if b then c else d`

For conditionals, there are two cases to consider,
depending on whether the condition evaluates to true or to false.

````
  ⇒-if-tt : ∀ {b s c s' d} →
    ⟦ b ⟧B s ≡ true →
    c , s ⇒ s' →
    if b then c else d , s ⇒ s'
    
  ⇒-if-ff : ∀ {b s c s' d} →
    ⟦ b ⟧B s ≡ false →
    d , s ⇒ s' →
    if b then c else d , s ⇒ s'
````

### `while b do: c`

Similarly, for while loops there are two cases to consider.

````
  ⇒-while-tt : ∀ {b s c s' s''} →
    ⟦ b ⟧B s ≡ true →
    c , s ⇒ s' →
    while b do: c , s' ⇒ s'' →
    while b do: c , s ⇒ s''
  
  ⇒-while-ff : ∀ {b s c} →
    ⟦ b ⟧B s ≡ false →
    while b do: c , s ⇒ s
````

This concludes the definition of the operational semantics ` c , s ⇒ s'`.

## **Exercise**: `do: c while b`

Extend the syntax and semantics of imperative programs
by adding a new construct

````
-- do: c while b
````

where `c` is a command and `b` a boolean expression.
The informal semantics of the do-while construct
is to first execute `c`,
and then evaluate the guard `b`:
If it evaluates to true, then we repeat the process,
otherwise we exit the loop.

## **Exercise**: Determinism

Since we cannot encode the semantics of imperative programs as a function,
we immediately lose some properties which are for free for functions.

One example is deterministic evaluation.
Consequently, we need to prove separately
that evaluation of programs is deterministic.

Show that evaluation of imperative programs is deterministic.

In [ ]:
{-# OPTIONS --allow-unsolved-metas #-}
module code.imp.det where
open import code.imp public

-- convenient to rule out some impossible cases.
false≡true : {A : Set} → false ≡ true → A
false≡true ()

⇒-det : ∀ {c s s' s''} → c , s ⇒ s' → c , s ⇒ s'' → s' ≡ s''
⇒-det ⇒-skip ⇒-skip = ?
⇒-det ⇒-assign ⇒-assign = ?
⇒-det (⇒-seq der1 der2) (⇒-seq der3 der4) = ?
⇒-det (⇒-if-tt _ der1) (⇒-if-tt _ der2) = ?
⇒-det (⇒-if-tt x _) (⇒-if-ff y _) = ?
⇒-det (⇒-if-ff x _) (⇒-if-tt y _) = ?
⇒-det (⇒-if-ff _ der1) (⇒-if-ff _ der2) = ?
⇒-det (⇒-while-tt x der1 der2) (⇒-while-tt y der3 der4) = ?
⇒-det (⇒-while-tt x _ _) (⇒-while-ff y) = ?
⇒-det (⇒-while-ff x) (⇒-while-tt y _ _) = ?
⇒-det (⇒-while-ff _) (⇒-while-ff _) = ?

?0 : s ≡ s
?1 : s [ x ↦ ⟦ e ⟧A s ] ≡ s [ x ↦ ⟦ e ⟧A s ]
?2 : s' ≡ s''
?3 : s' ≡ s''
?4 : s' ≡ s''
?5 : s' ≡ s''
?6 : s' ≡ s''
?7 : s' ≡ s''
?8 : s' ≡ s
?9 : s ≡ s''
?10 : s ≡ s


## **Exercise**: `loop`

Show that the program `loop` introduced above never stops.

In [ ]:
module code.imp.loop where
open import code.imp.det public

loop-⊥ : ∀ {s s'} → ¬ (loop , s ⇒ s')
loop-⊥ = ?

?0 : ¬ loop , s ⇒ s'


## Small-steps operational semantics

````
module code.imp.small-steps where
open import code.imp.det public
open import code.product public
````

We provide an alternative small-steps semantics for imperative programs.

### Configurations

````
Conf = Cmd ∧ State -- Cartesian product
````

We treat configurations of the form `skip , s` as final,
hence there is no rule for the `skip` command.

````
infix 3 _↝_
data _↝_ : Conf → Conf → Set where
````

### `x ≔ e`

In the rule for assignment,
we just evaluate the arithmetic expression `e`
and update the state accordingly.

(We could have used the operational semantics of arithmetic expressions here,
but we avoid it for simplicity.
A similar remark applies to boolean expressions below.)

````
  ↝-assign : ∀ {x e s} →
    x ≔ e , s ↝ skip , s [ x ↦ ⟦ e ⟧A s ]
````

### `c ⨟ d`

In the case of sequencing `c ⨟ d`,
where are two cases.

In the case `↝-seq-left`, we evaluate one step of `c`.
In the case `↝-seq-right`, `c` has finished and we continue with `d`.

````
  ↝-seq-left : ∀ {c s c' s' d} →
    c , s ↝ c' , s' →
    c ⨟ d , s ↝ c' ⨟ d , s'
    
  ↝-seq-right : ∀ {d s} →
    skip ⨟ d , s ↝ d , s
````

### `if b then c else d`

The conditional has two symmetric cases,
depending on whether the condition evaluates to `true` or `false`.

````
  ↝-if-tt : ∀ {b s c d} →
    ⟦ b ⟧B s ≡ true →
    if b then c else d , s ↝ c , s
    
  ↝-if-ff : ∀ {b s c d} →
    ⟦ b ⟧B s ≡ false →
    if b then c else d , s ↝ d , s
````

### `while b do: c`

The while looping construct also has two cases.
If the condition `b` evaluates to true,
then the command rewrites to `c ⨟ while b do: c`.
Otherwise, it terminates rewriting to `skip`.

````
  ↝-while-tt : ∀ {b c s} →
    ⟦ b ⟧B s ≡ true →
    while b do: c , s ↝ c ⨟ while b do: c , s
    
  ↝-while-ff : ∀ {b c s} →
    ⟦ b ⟧B s ≡ false →
    while b do: c , s ↝ skip , s
````

## Transitive closure `↝*`

In order to formalise this, we first need to be able to combine many small steps together, i.e., we take the transitive closure of `_,_↝_,_`.

````
infix 3 _↝*_
data _↝*_ : Conf → Conf → Set where
  stop : ∀ {x} → x ↝* x
  one : ∀ {x y z} → x ↝ y → y ↝* z → x ↝* z
````

### **Exercise**: Transitivity

Show that the relation `↝*` on configurations is indeed transitive.
*Hint*: Do induction on `der1`.

In [182]:
{-# OPTIONS --allow-unsolved-metas #-}
module code.imp.small-steps.trans where
open import code.imp.small-steps public

↝*-trans : ∀ {x y z} → x ↝* y → y ↝* z → x ↝* z
↝*-trans der1 der2 = ?

?0 : x ↝* z


### Notation for transitive closure

````
module code.imp.small-steps.chains where
open import code.imp.small-steps.trans public
````

The following suggestive notation will be useful for simplyfying carrying out proofs involving `↝*`.
They are entirely analogous to the chain of equalities constructed with `≡⟨⟩`.

````
infix  1 start_
infixr 2 _↝*⟨⟩_ _↝⟨_⟩_ _↝*⟨_⟩_
infix  3 _end

start_ : ∀ {x y} → x ↝* y → x ↝* y
start x↝*y = x↝*y

_↝*⟨⟩_ : ∀ x {y} → x ↝* y → x ↝* y
x ↝*⟨⟩ x↝*y = x↝*y

_↝⟨_⟩_ : ∀ x {y z} → x ↝ y → y ↝* z → x ↝* z
x ↝⟨ x↝y ⟩ y↝*z = one x↝y y↝*z

_↝*⟨_⟩_ : ∀ x {y z} → x ↝* y → y ↝* z → x ↝* z
x ↝*⟨ x↝*y ⟩ y↝*z = ↝*-trans x↝*y y↝*z

_end : ∀ x → x ↝* x
x end = stop
````

### Two lemmas about `⨟`

Our objective is to show that small steps and big steps semantics agree.

Before showing this,
we need to prove two lemmas (converse of each other)
connecting sequencing `⨟` and `↝*`.

#### **Exercise**: First lemma

In [185]:
module code.imp.small-steps.lemma1 where
open import code.imp.small-steps.chains public

⨟-lemma-1 : ∀ {c d s s'} →
  c ⨟ d , s ↝* skip , s' →
  ∃[ s'' ] c , s ↝* skip , s'' ∧ d , s'' ↝* skip , s'
⨟-lemma-1 (one (↝-seq-left step) der) = ?
⨟-lemma-1 {s = s} (one ↝-seq-right der) = ?

?0
  : thereExists (λ s'' → c , s ↝* skip , s'' ∧ d , s'' ↝* skip , s')
?1
  : thereExists
    (λ s'' → skip , s ↝* skip , s'' ∧ d , s'' ↝* skip , s')


#### **Exercise**: Second lemma

In [187]:
module code.imp.small-steps.lemma2 where
open import code.imp.small-steps.chains public

⨟-lemma-2 : ∀ {c d s s' s''} →
  c , s ↝* skip , s'' →
  d , s'' ↝* skip , s' →
  c ⨟ d , s ↝* skip , s'
⨟-lemma-2 stop der2 = ?
⨟-lemma-2 {c} {d} {s} {s'} {s''} (one {y = c' , s'''} step der1) der2 = ?

?0 : skip ⨟ d , s ↝* skip , s'
?1 : c ⨟ d , s ↝* skip , s'
